In [ ]:
"""
Simulación Numérica de Flujo Lid-Driven Cavity 

Este código implementa una simulación numérica del flujo de un fluido incompresible
Lid-Driven Cavity utilizando el método de diferencias finitas. Se resuelven
las ecuaciones de Navier-Stokes mediante un esquema de proyección temporal. 
Asimismo se muestran dos formas de predicción de la ssoluciones de dicho sistema con PINNs e I-PINNs

Características principales:
- Resuelve las ecuaciones por medio de diferencias fintas
- Genera una predicción PINNs del mismo sistema
- Genera una predicción I-PINNs del mismo sistema con mejor resolución de predicción.
- Integra ecuaciones en el proceso de aprendizaje
- Utiliza datos de simulación directa como guía
- Impone condiciones de contorno físicamente relevantes
- Escala las derivadas para mejorar el entrenamiento

Autor: Leonardo Córdova
Fecha: 09.12.2024
"""

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

N_POINTS = 50
DOMAIN_SIZE = 1.0
N_ITERATIONS = 300
TIME_STEP_LENGTH = 0.001
RE = 10
KINEMATIC_VISCOSITY = 1/RE
DENSITY = 1.0
HORIZONTAL_VELOCITY_TOP = 1.0

N_PRESSURE_POISSON_ITERATIONS = 50
STABILITY_SAFETY_FACTOR = 0.5

def main():
    element_length = DOMAIN_SIZE / (N_POINTS - 1)
    x = np.linspace(0.0, DOMAIN_SIZE, N_POINTS)
    y = np.linspace(0.0, DOMAIN_SIZE, N_POINTS)

    X, Y = np.meshgrid(x, y)

    u_prev = np.zeros_like(X)
    v_prev = np.zeros_like(X)
    p_prev = np.zeros_like(X)

    def central_difference_x(f):
        diff = np.zeros_like(f)
        diff[1:-1, 1:-1] = (
            f[1:-1, 2:  ]
            -
            f[1:-1, 0:-2]
        ) / (
            2 * element_length
        )
        return diff
    
    def central_difference_y(f):
        diff = np.zeros_like(f)
        diff[1:-1, 1:-1] = (
            f[2:  , 1:-1]
            -
            f[0:-2, 1:-1]
        ) / (
            2 * element_length
        )
        return diff
    
    def laplace(f):
        diff = np.zeros_like(f)
        diff[1:-1, 1:-1] = (
            f[1:-1, 0:-2]
            +
            f[0:-2, 1:-1]
            -
            4
            *
            f[1:-1, 1:-1]
            +
            f[1:-1, 2:  ]
            +
            f[2:  , 1:-1]
        ) / (
            element_length**2
        )
        return diff
    

    maximum_possible_time_step_length = (
        0.5 * element_length**2 / KINEMATIC_VISCOSITY
    )
    if TIME_STEP_LENGTH > STABILITY_SAFETY_FACTOR * maximum_possible_time_step_length:
        raise RuntimeError("Stability is not guarenteed")

    
    for _ in tqdm(range(N_ITERATIONS)):
        d_u_prev__d_x = central_difference_x(u_prev)
        d_u_prev__d_y = central_difference_y(u_prev)
        d_v_prev__d_x = central_difference_x(v_prev)
        d_v_prev__d_y = central_difference_y(v_prev)
        laplace__u_prev = laplace(u_prev)
        laplace__v_prev = laplace(v_prev)

        # Perform a tentative step by solving the momentum equation without the
        # pressure gradient
        u_tent = (
            u_prev
            +
            TIME_STEP_LENGTH * (
                -
                (
                    u_prev * d_u_prev__d_x
                    +
                    v_prev * d_u_prev__d_y
                )
                +
                KINEMATIC_VISCOSITY * laplace__u_prev
            )
        )
        v_tent = (
            v_prev
            +
            TIME_STEP_LENGTH * (
                -
                (
                    u_prev * d_v_prev__d_x
                    +
                    v_prev * d_v_prev__d_y
                )
                +
                KINEMATIC_VISCOSITY * laplace__v_prev
            )
        )

        # Velocity Boundary Conditions: Homogeneous Dirichlet BC everywhere
        # except for the horizontal velocity at the top, which is prescribed
        u_tent[0, :] = 0.0
        u_tent[:, 0] = 0.0
        u_tent[:, -1] = 0.0
        u_tent[-1, :] = HORIZONTAL_VELOCITY_TOP
        v_tent[0, :] = 0.0
        v_tent[:, 0] = 0.0
        v_tent[:, -1] = 0.0
        v_tent[-1, :] = 0.0


        d_u_tent__d_x = central_difference_x(u_tent)
        d_v_tent__d_y = central_difference_y(v_tent)

        # Compute a pressure correction by solving the pressure-poisson equation
        rhs = (
            DENSITY / TIME_STEP_LENGTH
            *
            (
                d_u_tent__d_x
                +
                d_v_tent__d_y
            )
        )

        for _ in range(N_PRESSURE_POISSON_ITERATIONS):
            p_next = np.zeros_like(p_prev)
            p_next[1:-1, 1:-1] = 1/4 * (
                +
                p_prev[1:-1, 0:-2]
                +
                p_prev[0:-2, 1:-1]
                +
                p_prev[1:-1, 2:  ]
                +
                p_prev[2:  , 1:-1]
                -
                element_length**2
                *
                rhs[1:-1, 1:-1]
            )

            # Pressure Boundary Conditions: Homogeneous Neumann Boundary
            # Conditions everywhere except for the top, where it is a
            # homogeneous Dirichlet BC
            p_next[:, -1] = p_next[:, -2]
            p_next[0,  :] = p_next[1,  :]
            p_next[:,  0] = p_next[:,  1]
            p_next[-1, :] = 0.0

            p_prev = p_next
        

        d_p_next__d_x = central_difference_x(p_next)
        d_p_next__d_y = central_difference_y(p_next)

        # Correct the velocities such that the fluid stays incompressible
        u_next = (
            u_tent
            -
            TIME_STEP_LENGTH / DENSITY
            *
            d_p_next__d_x
        )
        v_next = (
            v_tent
            -
            TIME_STEP_LENGTH / DENSITY
            *
            d_p_next__d_y
        )

        # Velocity Boundary Conditions: Homogeneous Dirichlet BC everywhere
        # except for the horizontal velocity at the top, which is prescribed
        u_next[0, :] = 0.0
        u_next[:, 0] = 0.0
        u_next[:, -1] = 0.0
        u_next[-1, :] = HORIZONTAL_VELOCITY_TOP
        v_next[0, :] = 0.0
        v_next[:, 0] = 0.0
        v_next[:, -1] = 0.0
        v_next[-1, :] = 0.0


        # Advance in time
        u_prev = u_next
        v_prev = v_next
        p_prev = p_next

    # Update fields
        u = u_next
        v = v_next

    ####################################################3333
    index_x_05 = np.argmin(np.abs(x - (0.5)))
    index_y_07 = np.argmin(np.abs(y - (0.7)))
# Guardar los datos corregidos
    np.savetxt('u_x05_y_DF_LID.dat', u[:, index_x_05], fmt='%.6f')
    np.savetxt('v_x_y07_DF_LID.dat', v[index_y_07, :], fmt='%.6f')
########################################################3
    
    # Save the velocity field (u_next and v_next) to a file
    np.savez("velocity_field.npz", u=u_next, v=v_next)
    
    # Plotting
    plt.style.use("default")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot streamlines
    speed = np.sqrt(u_next**2 + v_next**2)
    skip = 2
    # 1. Contour plot for background (velocity magnitude)
    contour = ax1.contourf(X, Y, speed, 40, cmap='jet', alpha=0.7)  # alpha for transparency

    # Crear la visualización del campo de velocidades
    quiver = ax2.quiver(X[::skip, ::skip], Y[::skip, ::skip], 
                       u_next[::skip, ::skip], v_next[::skip, ::skip],
                       speed[::skip, ::skip],  # Color basado en la magnitud
                       scale=2,  # Reducido para flechas más grandes
                       width=0.005,  # Ancho de las flechas
                       headwidth=3,  # Ancho de la punta de la flecha
                       headlength=5,  # Longitud de la punta de la flecha
                       headaxislength=4.5,  # Longitud del eje de la punta
                       cmap='jet'  # Colormap para la magnitud
                       )
        
    # Agregar una barra de color
    cbar = fig.colorbar(quiver, ax=ax1, label='Velocidad')
    
    ax1.streamplot(X, Y, u_next, v_next, density=1.5, color='white')
    #ax1.set_title('Streamlines')
    ax1.set_aspect('equal')
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    ax1.set_xlabel('X', rotation=0)
    ax1.set_ylabel('Y', rotation=0)
    ax1.grid(False)

    speed = np.sqrt(u_next**2 + v_next**2)
    
    ax2.set_title('Velocity Field')
    ax2.set_aspect('equal')
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 1)
    ax2.set_xlabel('X', rotation=0)
    ax2.set_ylabel('Y', rotation=0)
    ax2.grid(False)
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()


In [ ]:
import deepxde as dde
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from deepxde.callbacks import CallbackList

# Parámetros de simulación
RE = 10.0  # Reynolds number
HORIZONTAL_VELOCITY_TOP = 1.0  # Velocidad de la tapa superior
DOMAIN_SIZE = 1.0  # Tamaño del dominio

# Definir la geometría: Cavidad cuadrada [0,1]x[0,1]
geom = dde.geometry.Rectangle([0, 0], [DOMAIN_SIZE, DOMAIN_SIZE])

# Condiciones de contorno
def boundary_top(x, on_boundary):
    return on_boundary and np.isclose(x[1], DOMAIN_SIZE)

def boundary_other(x, on_boundary):
    return on_boundary and not np.isclose(x[1], DOMAIN_SIZE)

# Condición de velocidad en la tapa superior
bc_top_u = dde.DirichletBC(geom, lambda x: HORIZONTAL_VELOCITY_TOP, boundary_top, component=0)
bc_top_v = dde.DirichletBC(geom, lambda x: 0, boundary_top, component=1)

# Condiciones de no deslizamiento en las otras paredes
bc_other_u = dde.DirichletBC(geom, lambda x: 0, boundary_other, component=0)
bc_other_v = dde.DirichletBC(geom, lambda x: 0, boundary_other, component=1)

# Ecuaciones de Navier-Stokes
def pde(x, y):
    """
    x: coordenadas espaciales
    y: [u, v, p] velocidades y presión
    """
    u, v, p = y[:, 0:1], y[:, 1:2], y[:, 2:3]
    
    # Gradientes de primer orden
    du_x = dde.grad.jacobian(y, x, i=0, j=0)
    du_y = dde.grad.jacobian(y, x, i=0, j=1)
    dv_x = dde.grad.jacobian(y, x, i=1, j=0)
    dv_y = dde.grad.jacobian(y, x, i=1, j=1)
    dp_x = dde.grad.jacobian(y, x, i=2, j=0)
    dp_y = dde.grad.jacobian(y, x, i=2, j=1)

    # Términos de segundo orden
    du_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    du_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)
    dv_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
    dv_yy = dde.grad.hessian(y, x, component=1, i=1, j=1)

    # Ecuaciones de Navier-Stokes
    momentum_x = (
        u * du_x + v * du_y + dp_x - 
        (1.0/RE) * (du_xx + du_yy)
    )

    momentum_y = (
        u * dv_x + v * dv_y + dp_y - 
        (1.0/RE) * (dv_xx + dv_yy)
    )

    continuity = du_x + dv_y

    return [momentum_x, momentum_y, continuity]

# Crear el problema
data = dde.data.PDE(
    geom,
    pde,
    [bc_top_u, bc_top_v, bc_other_u, bc_other_v],
    num_domain=2500,
    num_boundary=500,
    solution=None,
    num_test=100
)

# Definir la arquitectura de la red neuronal
layers = [2] + [50] * 4 + [3]  # Red más profunda para capturar la complejidad
activation = "tanh"
initializer = "Glorot uniform"

# Crear la red neuronal
net = dde.maps.FNN(layers, activation, initializer)

# Crear el modelo
model = dde.Model(data, net)

# Visualización de los puntos de entrenamiento
plt.figure(figsize=(10, 8))
plt.scatter(data.train_x_all[:, 0], data.train_x_all[:, 1], s=0.5)
plt.xlabel("x")
plt.ylabel("y")
plt.title("Puntos de entrenamiento en el dominio")
plt.show()



#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Compilar y entrenar
model.compile("adam", lr=1e-3)
losshistory, train_state = model.train(iterations=20000)

# Refinamiento con L-BFGS
model.compile("L-BFGS")
losshistory, train_state = model.train()


def plot_results(model):
    # Crear malla para visualización
    n_points = 100
    x = np.linspace(0, DOMAIN_SIZE, n_points)
    y = np.linspace(0, DOMAIN_SIZE, n_points)
    X, Y = np.meshgrid(x, y)
    X_flat = X.flatten()[:, None]
    Y_flat = Y.flatten()[:, None]
    points = np.hstack((X_flat, Y_flat))

    # Predecir velocidades y presión
    predictions = model.predict(points)
    u = predictions[:, 0].reshape(X.shape)
    v = predictions[:, 1].reshape(X.shape)
    p = predictions[:, 2].reshape(X.shape)

    # Calcular magnitud de velocidad
    speed = np.sqrt(u**2 + v**2)

    # Crear la figura con tres subplots
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

    # Plot 1: Líneas de corriente
    streamplot = ax1.streamplot(X, Y, u, v, density=1.5, color='blue')
    ax1.set_title('Streamlines')
    ax1.set_aspect('equal')
    ax1.set_xlim(0, DOMAIN_SIZE)
    ax1.set_ylim(0, DOMAIN_SIZE)
    ax1.grid(True)

    # Plot 2: Campo de velocidades
    skip = 5
    quiver = ax2.quiver(X[::skip, ::skip], Y[::skip, ::skip],
                       u[::skip, ::skip], v[::skip, ::skip],
                       speed[::skip, ::skip],
                       scale=2,
                       width=0.005,
                       cmap='viridis')
    
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(quiver, cax=cax, label='Velocity magnitude')
    
    ax2.set_title('Velocity Field')
    ax2.set_aspect('equal')
    ax2.set_xlim(0, DOMAIN_SIZE)
    ax2.set_ylim(0, DOMAIN_SIZE)
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

# Mostrar resultados
plot_results(model)#, geom)



###########################################################3333


def plot_results(model):
    # Crear malla para visualización
    n_points = 50 
    x = np.linspace(0, DOMAIN_SIZE, n_points)
    y = np.linspace(0, DOMAIN_SIZE, n_points)
    X, Y = np.meshgrid(x, y)
    X_flat = X.flatten()[:, None]
    Y_flat = Y.flatten()[:, None]
    points = np.hstack((X_flat, Y_flat))
    
    # Predecir velocidades y presión
    predictions = model.predict(points)
    u = predictions[:, 0].reshape(X.shape)
    v = predictions[:, 1].reshape(X.shape)
    p = predictions[:, 2].reshape(X.shape)
    
    # Crear la figura con tres subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
    
    # Calcular magnitud de velocidad
    speed = np.sqrt(u**2 + v**2)
    
    # Plot 2: Campo de velocidades
    skip = 5
    contour = ax1.contourf(X, Y, speed, 40, cmap='jet', alpha=0.7)
    
    # Plot 1: Líneas de corriente
    streamplot = ax1.streamplot(X, Y, u, v, density=1.5, color='white')
    ax1.set_aspect('equal')
    ax1.set_xlim(0, DOMAIN_SIZE)
    ax1.set_ylim(0, DOMAIN_SIZE)
    ax1.set_xlabel('X', rotation=0)
    ax1.set_ylabel('Y', rotation=0)
    ax1.grid(True)
    
    quiver = ax2.quiver(X[::skip, ::skip], Y[::skip, ::skip],
                       u[::skip, ::skip], v[::skip, ::skip],
                       speed[::skip, ::skip],
                       scale=2,
                       width=0.005,
                       cmap='jet')
    
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.3)
    
    # Modificar el colorbar con propiedades personalizadas
    cbar = plt.colorbar(quiver, cax=cax)
    cbar.set_label('Velocidad', size=13)  # Ajustar tamaño de la etiqueta
    cbar.ax.tick_params(labelsize=12)     # Ajustar tamaño de los números
    
    ax2.set_title('Velocity Field')
    ax2.set_aspect('equal')
    ax2.set_xlim(0, DOMAIN_SIZE)
    ax2.set_ylim(0, DOMAIN_SIZE)
    ax2.set_xlabel('X', rotation=0)
    ax2.set_ylabel('Y', rotation=0)
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

# Mostrar resultados
plot_results(model)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import deepxde as dde
import tensorflow as tf
from tqdm import tqdm



# Simulation parameters (shared between simulations)
N_POINTS = 41
DOMAIN_SIZE = 1.0
N_ITERATIONS = 500
TIME_STEP_LENGTH = 0.001
RE = 10
KINEMATIC_VISCOSITY = 1/RE
DENSITY = 1.0
HORIZONTAL_VELOCITY_TOP = 1.0

N_PRESSURE_POISSON_ITERATIONS = 50
STABILITY_SAFETY_FACTOR = 0.5

def run_direct_simulation():
    """Original lid driven simulation to generate training data"""
    # Create mesh
    #element_length = DOMAIN_SIZE / (N_POINTS - 1)
    #x = np.linspace(-DOMAIN_SIZE/2, DOMAIN_SIZE/2, N_POINTS)
    #y = np.linspace(-DOMAIN_SIZE/2, DOMAIN_SIZE/2, N_POINTS)
    #X, Y = np.meshgrid(x, y)
    element_length = DOMAIN_SIZE / (N_POINTS - 1)
    x = np.linspace(0.0, DOMAIN_SIZE, N_POINTS)
    y = np.linspace(0.0, DOMAIN_SIZE, N_POINTS)
    X, Y = np.meshgrid(x, y)
    # Initialize fields
    u = np.zeros_like(X)
    v = np.zeros_like(X)
    p = np.zeros_like(X)
    
    u_prev = np.zeros_like(X)
    v_prev = np.zeros_like(X)
    p_prev = np.zeros_like(X)

    def central_difference_x(f):
        diff = np.zeros_like(f)
        diff[1:-1, 1:-1] = (
            f[1:-1, 2:  ]
            -
            f[1:-1, 0:-2]
        ) / (
            2 * element_length
        )
        return diff
    
    def central_difference_y(f):
        diff = np.zeros_like(f)
        diff[1:-1, 1:-1] = (
            f[2:  , 1:-1]
            -
            f[0:-2, 1:-1]
        ) / (
            2 * element_length
        )
        return diff
    
    def laplace(f):
        diff = np.zeros_like(f)
        diff[1:-1, 1:-1] = (
            f[1:-1, 0:-2]
            +
            f[0:-2, 1:-1]
            -
            4
            *
            f[1:-1, 1:-1]
            +
            f[1:-1, 2:  ]
            +
            f[2:  , 1:-1]
        ) / (
            element_length**2
        )
        return diff
    

    maximum_possible_time_step_length = (
        0.5 * element_length**2 / KINEMATIC_VISCOSITY
    )
    if TIME_STEP_LENGTH > STABILITY_SAFETY_FACTOR * maximum_possible_time_step_length:
        raise RuntimeError("Stability is not guarenteed")

    
    for _ in tqdm(range(N_ITERATIONS)):
        d_u_prev__d_x = central_difference_x(u_prev)
        d_u_prev__d_y = central_difference_y(u_prev)
        d_v_prev__d_x = central_difference_x(v_prev)
        d_v_prev__d_y = central_difference_y(v_prev)
        laplace__u_prev = laplace(u_prev)
        laplace__v_prev = laplace(v_prev)

        # Perform a tentative step by solving the momentum equation without the
        # pressure gradient
        u_tent = (
            u_prev
            +
            TIME_STEP_LENGTH * (
                -
                (
                    u_prev * d_u_prev__d_x
                    +
                    v_prev * d_u_prev__d_y
                )
                +
                KINEMATIC_VISCOSITY * laplace__u_prev
            )
        )
        v_tent = (
            v_prev
            +
            TIME_STEP_LENGTH * (
                -
                (
                    u_prev * d_v_prev__d_x
                    +
                    v_prev * d_v_prev__d_y
                )
                +
                KINEMATIC_VISCOSITY * laplace__v_prev
            )
        )

        # Velocity Boundary Conditions: Homogeneous Dirichlet BC everywhere
        # except for the horizontal velocity at the top, which is prescribed
        u_tent[0, :] = 0.0
        u_tent[:, 0] = 0.0
        u_tent[:, -1] = 0.0
        u_tent[-1, :] = HORIZONTAL_VELOCITY_TOP
        v_tent[0, :] = 0.0
        v_tent[:, 0] = 0.0
        v_tent[:, -1] = 0.0
        v_tent[-1, :] = 0.0


        d_u_tent__d_x = central_difference_x(u_tent)
        d_v_tent__d_y = central_difference_y(v_tent)

        # Compute a pressure correction by solving the pressure-poisson equation
        rhs = (
            DENSITY / TIME_STEP_LENGTH
            *
            (
                d_u_tent__d_x
                +
                d_v_tent__d_y
            )
        )

        for _ in range(N_PRESSURE_POISSON_ITERATIONS):
            p_next = np.zeros_like(p_prev)
            p_next[1:-1, 1:-1] = 1/4 * (
                +
                p_prev[1:-1, 0:-2]
                +
                p_prev[0:-2, 1:-1]
                +
                p_prev[1:-1, 2:  ]
                +
                p_prev[2:  , 1:-1]
                -
                element_length**2
                *
                rhs[1:-1, 1:-1]
            )

            # Pressure Boundary Conditions: Homogeneous Neumann Boundary
            # Conditions everywhere except for the top, where it is a
            # homogeneous Dirichlet BC
            p_next[:, -1] = p_next[:, -2]
            p_next[0,  :] = p_next[1,  :]
            p_next[:,  0] = p_next[:,  1]
            p_next[-1, :] = 0.0

            p_prev = p_next
        

        d_p_next__d_x = central_difference_x(p_next)
        d_p_next__d_y = central_difference_y(p_next)

        # Correct the velocities such that the fluid stays incompressible
        u_next = (
            u_tent
            -
            TIME_STEP_LENGTH / DENSITY
            *
            d_p_next__d_x
        )
        v_next = (
            v_tent
            -
            TIME_STEP_LENGTH / DENSITY
            *
            d_p_next__d_y
        )

        # Velocity Boundary Conditions: Homogeneous Dirichlet BC everywhere
        # except for the horizontal velocity at the top, which is prescribed
        u_next[0, :] = 0.0
        u_next[:, 0] = 0.0
        u_next[:, -1] = 0.0
        u_next[-1, :] = HORIZONTAL_VELOCITY_TOP
        v_next[0, :] = 0.0
        v_next[:, 0] = 0.0
        v_next[:, -1] = 0.0
        v_next[-1, :] = 0.0


        # Advance in time
        u_prev = u_next
        v_prev = v_next
        p_prev = p_next
        # Update fields
        u = u_next
        v = v_next

    return X, Y, u, v

class LidDrivenSolver:

    def __init__(self, domain_size=1.0, n_points=41, re=10.0):
        self.domain_size = domain_size
        self.n_points = n_points
        self.re = re
        self.setup_geometry()
        
    def setup_geometry(self):
        """Initialize spatial domain and fields"""
        self.x = np.linspace(0, self.domain_size, self.n_points)
        self.y = np.linspace(0, self.domain_size, self.n_points)
        self.X, self.Y = np.meshgrid(self.x, self.y)
        self.dx = self.domain_size / (self.n_points - 1)
        
    def create_training_data(self, u, v, n_samples=1000):  # Reducir n_samples por defecto
        """Create training data with improved sampling"""
        print("Shape of u:", u.shape)
        print("Shape of v:", v.shape)
        print("Shape of X:", self.X.shape)
        print("Shape of Y:", self.Y.shape)

        # Calculate gradients for importance sampling
        du_dx = np.gradient(u, self.dx, axis=1)
        du_dy = np.gradient(u, self.dx, axis=0)
        dv_dx = np.gradient(v, self.dx, axis=1)
        dv_dy = np.gradient(v, self.dx, axis=0)

        # Calculate importance weights
        importance = np.sqrt(du_dx**2 + du_dy**2 + dv_dx**2 + dv_dy**2)
        importance = importance / importance.sum()

        # Flatten arrays consistently
        points = np.column_stack((self.X.flatten(), self.Y.flatten()))
        values = np.column_stack((u.flatten(), v.flatten()))

        # Flatten importance consistently with points and values
        weights = importance.flatten()
        weights = np.abs(weights)
        weights = weights / weights.sum()

        # Ensure weights are not zero and we don't sample more points than available
        nonzero_weights = weights > 0
        valid_indices = np.where(nonzero_weights)[0]
        n_available = len(valid_indices)
        n_samples = min(n_samples, n_available)

        # Weighted sampling
        idx = np.random.choice(
            valid_indices,
            size=n_samples,
            replace=False,
            p=weights[nonzero_weights] / weights[nonzero_weights].sum()
        )

        train_points = points[idx]
        train_values = values[idx]

        return train_points, train_values[:, 0], train_values[:, 1]

    def create_neural_network(self, train_points, train_u, train_v):
        """Create an improved neural network architecture"""
        # Define geometry
        # Definir la geometría: Cavidad cuadrada [0,1]x[0,1]
        geom = dde.geometry.Rectangle([0, 0], [DOMAIN_SIZE, DOMAIN_SIZE])
        
######################################################################
      #  def boundary(x, on_boundary):
      #      return on_boundary

        # Condiciones de contorno
        def boundary_top(x, on_boundary):
            return on_boundary and np.isclose(x[1], DOMAIN_SIZE)

        def boundary_other(x, on_boundary):
            return on_boundary and not np.isclose(x[1], DOMAIN_SIZE)

        # Condición de velocidad en la tapa superior
        bc_top_u = dde.DirichletBC(geom, lambda x: HORIZONTAL_VELOCITY_TOP, boundary_top, component=0)
        bc_top_v = dde.DirichletBC(geom, lambda x: 0, boundary_top, component=1)

        # Condiciones de no deslizamiento en las otras paredes
        bc_other_u = dde.DirichletBC(geom, lambda x: 0, boundary_other, component=0)
        bc_other_v = dde.DirichletBC(geom, lambda x: 0, boundary_other, component=1)
      
        # Improved observations with noise handling
        observe_u = dde.PointSetBC(train_points, train_u.reshape(-1, 1), component=0)
        observe_v = dde.PointSetBC(train_points, train_v.reshape(-1, 1), component=1)
        
        def pde(x, y):
            """Enhanced PDE implementation with scaled derivatives"""
            u, v, p = y[:, 0:1], y[:, 1:2], y[:, 2:3]
            
            # Scaled derivatives
            scale = 1.0 / self.domain_size
            
            # First order terms
            du_x = dde.grad.jacobian(y, x, i=0, j=0) * scale
            du_y = dde.grad.jacobian(y, x, i=0, j=1) * scale
            dv_x = dde.grad.jacobian(y, x, i=1, j=0) * scale
            dv_y = dde.grad.jacobian(y, x, i=1, j=1) * scale
            dp_x = dde.grad.jacobian(y, x, i=2, j=0) * scale
            dp_y = dde.grad.jacobian(y, x, i=2, j=1) * scale
            
            # Second order terms
            du_xx = dde.grad.hessian(y, x, component=0, i=0, j=0) * scale**2
            du_yy = dde.grad.hessian(y, x, component=0, i=1, j=1) * scale**2
            dv_xx = dde.grad.hessian(y, x, component=1, i=0, j=0) * scale**2
            dv_yy = dde.grad.hessian(y, x, component=1, i=1, j=1) * scale**2
            
           
            # Modified equations with proper scaling
            momentum_x = (u * du_x + v * du_y + dp_x - 
                        (1.0/self.re) * (du_xx + du_yy))
            
            momentum_y = (u * dv_x + v * dv_y + dp_y - 
                        (1.0/self.re) * (dv_xx + dv_yy))
            
            continuity = du_x + dv_y
            
            return [momentum_x, momentum_y, continuity]
        
        data = dde.data.PDE(
            geom,
            pde,
            [bc_top_u,bc_top_v,bc_other_u,bc_other_v, observe_u, observe_v],
            num_domain=3000,
            num_boundary=400,
            anchors=train_points,
            num_test=500
        )
        
        # Create the network
        layers = [2] + [100] * 3 + [3]
        net = dde.maps.FNN(layers, "tanh", "Glorot normal")
        
        # Create the model
        model = dde.Model(data, net)
        
        # Advanced training strategy with corrected loss weights
      #  model.compile(
       #     "adam", 
       #     lr=1e-3,
       #     loss_weights=[1, 1, 1, 5, 5, 1, 1],  # Corrected weights for all 7 components
       #     decay=("cosine", 50000, 0.1)
       # )

        model.compile(
        "adam", 
        lr=1e-3,
        loss_weights=[1, 1, 1,  # PDEs: momentum_x, momentum_y, continuity
                     5, 5,      # wall conditions: u, v
                     5, 5,      # cylinder conditions: u, v
                     1, 1],     # observations: u, v
        decay=("cosine", 30000, 0.1)
    )

        return model

    def train_model(self, model, epochs=30000):
        """Enhanced training procedure"""
        checker = dde.callbacks.ModelCheckpoint(
            "model/model.ckpt", 
            save_better_only=True, 
            period=1000
        )
        
        # Multi-stage training
        print("Stage 1: Initial training...")
        model.train(epochs=epochs//2, display_every=1000, callbacks=[checker])
        
        print("Stage 2: Fine-tuning...")
        model.compile("L-BFGS-B")
        model.train()
        
        return model


def compare_results(X, Y, u, v, model):
    """Compare direct simulation with neural network predictions"""
    print("Comparing results...")
    
    # Prepare points for prediction
    x_flat = X.flatten()
    y_flat = Y.flatten()
    points = np.column_stack((x_flat, y_flat))
    
    # Get predictions
    predictions = model.predict(points)
    u_pred = predictions[:, 0].reshape(X.shape)
    v_pred = predictions[:, 1].reshape(X.shape)
    
    # Plot comparisons
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 15))
    
    # Direct simulation results
    im1 = ax1.contourf(X, Y, u, levels=50, cmap='jet')
    plt.colorbar(im1, ax=ax1)
    ax1.set_title('U velocity (Direct)')
    
    im2 = ax2.contourf(X, Y, v, levels=50, cmap='jet')
    plt.colorbar(im2, ax=ax2)
    ax2.set_title('V velocity (Direct)')
    
    # Neural network results
    im3 = ax3.contourf(X, Y, u_pred, levels=50, cmap='jet')
    plt.colorbar(im3, ax=ax3)
    ax3.set_title('U velocity (Neural Network)')
    
    im4 = ax4.contourf(X, Y, v_pred, levels=50, cmap='jet')
    plt.colorbar(im4, ax=ax4)
    ax4.set_title('V velocity (Neural Network)')
    
    plt.tight_layout()
    plt.show()

    # Extract velocity profiles
    index_x_05 = np.argmin(np.abs(X[0, :] - (0.5)))
    index_y_07 = np.argmin(np.abs(Y[:, 0] - (0.7)))
    
    
    # Perfiles U
    plt.figure(figsize=(10, 5))
    plt.plot(Y[:, 0], u[:, index_x_05], '-', color='blue', label='U(x=0.5, y) Direct')
    plt.plot(Y[:, 0], u_pred[:, index_x_05], '--', color='blue', label='U(x=0.5, y) Predicted')
    plt.xlabel('y')
    plt.ylabel('U velocity')
    plt.title('Comparison of U velocity profiles')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Perfiles V
    plt.figure(figsize=(10, 5))
    plt.plot(X[index_y_07, :], v[index_y_07, :], '-', color='red', label='V(x, y=0.7) Direct')
    plt.plot(X[index_y_07, :], v_pred[index_y_07, :], '--', color='red', label='V(x, y=0.7) Predicted')
    plt.xlabel('x')
    plt.ylabel('V velocity')
    plt.title('Comparison of V velocity profiles')
    plt.legend()
    plt.grid(True)
    plt.show() 



def main():
    print("1. Initializing solver...")
    solver = LidDrivenSolver()
    
    print("2. Running direct simulation...")
    X, Y, u, v = run_direct_simulation()
    
    print("3. Creating training data...")
    train_points, train_u, train_v = solver.create_training_data(u, v)
    
    print("4. Creating neural network...")
    model = solver.create_neural_network(train_points, train_u, train_v)
    
    print("5. Training model...")
    trained_model = solver.train_model(model)
    
    print("6. Comparing results...")
    compare_results(X, Y, u, v, trained_model)

if __name__ == "__main__":
    main()